# Shot Boundary Detection with Window Functions and SigLip2

For more than 10000 rows, a T4 instance is recommended.

In [1]:
!pip install -q "daft[huggingface]" transformers numpy av yt-dlp

In [2]:
# General Parameters
MODEL_ID = "google/siglip2-base-patch16-512"
B, T, H, W, C = 2, 16, 288, 288, 3 # Batch Size, Clip Size (# frames), Height, Width, RGB
ROW_LIMIT = 20000

PATHS = [
    "https://www.youtube.com/watch?v=eYXDSuNpKTk", # Life after Apache Spark
]


MIN_SHOT_DURATION = 1.0 # seconds


model_col = f"img_emb_{MODEL_ID}"

In [3]:
import daft
from daft.functions import embed_image
from daft import col, lit, Window, DataType as dt

import numpy as np

In [4]:
df_frames = daft.read_video_frames(
    PATHS,
    image_height=H,
    image_width=W,
).limit(ROW_LIMIT).collect() # Materialize so we don't re-read from YT
df_frames.show(3)

pathUtf8,frame_indexInt64,frame_timeFloat64,frame_time_baseUtf8,frame_ptsInt64,frame_dtsInt64,frame_durationInt64,is_key_frameBoolean,dataImage[RGB; 288 x 288]
https://www.youtube.com/watch?v=eYXDSuNpKTk,1032,34.434434434434436,1/11988,412800,412800,400,false,""" />"
https://www.youtube.com/watch?v=eYXDSuNpKTk,1033,34.4678011344678,1/11988,413200,413200,400,false,""" />"
https://www.youtube.com/watch?v=eYXDSuNpKTk,1034,34.50116783450117,1/11988,413600,413600,400,false,""" />"


### Generate SigLip2 Embeddings

In [5]:
df_emb = df_frames.with_column(
    model_col,
    embed_image(
        df_frames["data"],
        model_name=MODEL_ID,
        provider="transformers",
    )
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

In [ ]:
df_emb = df_emb.collect()

/usr/local/lib/python3.12/dist-packages/daft/dashboard/__init__.py:91: UserWarning: Failed to broadcast metrics over http://0.0.0.0:3238/api/queries: HTTP Error 422: Unprocessable Entity
  warnings.warn(f"Failed to broadcast metrics over {url}: {e}")


🗡️ 🐟 UDF _ImageEmbedderExpression: 00:00 

🗡️ 🐟 InMemorySource: 00:00 

### Cosine Similarity between Frames using Window Functions

In [ ]:
w = Window().partition_by("path").order_by("frame_time")
w_cut = w.range_between(-0.3, Window.current_row)
w_dissolve = w.range_between(-1.0, Window.current_row)

df_shots = (
    df_emb
    .with_column("cos_dist", col(model_col).embedding.cosine_distance(col(model_col).lag(1).over(w)))
    .with_column("cos_dist_cut", col("cos_dist").mean().over(w_cut))
    .with_column("cos_dist_dissolve", col("cos_dist").mean().over(w_dissolve))
)


In [ ]:
df_shots.collect()

In [ ]:
import matplotlib.pyplot as plt

# Select the relevant columns and convert to pandas DataFrame for plotting
df_plot_multiple = df_shots.select("frame_time", "cos_dist", "cos_dist_cut", "cos_dist_dissolve").to_pandas()

# Plot the data
plt.figure(figsize=(12, 6))
plt.plot(df_plot_multiple["frame_time"], df_plot_multiple["cos_dist"], label="Cosine Distance", alpha=0.7)
plt.plot(df_plot_multiple["frame_time"], df_plot_multiple["cos_dist_cut"], label="Cosine Distance (Cut Window)", linewidth=2)
plt.plot(df_plot_multiple["frame_time"], df_plot_multiple["cos_dist_dissolve"], label="Cosine Distance (Dissolve Window)", linewidth=2)
plt.xlabel("Frame Time (seconds)")
plt.ylabel("Cosine Distance")
plt.title("Cosine Distance and Smoothed Cosine Distances over Time")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
THRESHOLD = 0.1
df_sbd = df_shots.with_column("is_cut_boundary", (col("cos_dist") >= THRESHOLD))

In [ ]:
df_sbd.where(df_sbd["is_cut_boundary"]).select("data").show()